In [12]:
import group_data as gd
import flylib
import numpy as np

In [13]:
%pylab inline
%config InlineBackend.figure_format = 'svg'
import pylab as plb
plb.rcParams['pdf.fonttype'] = 42

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [14]:
### summarize xcov data accross flies - just stripe fixation for now.

In [15]:
import h5py
trial_db = h5py.File('trial_db.hdf5','r')
xi = np.linspace(0,23,1150)
tlag = xi-xi[1150/2.0]
tlag = tlag[:1149]
muscle_list =  ['b1','b2','b3',
                'i1','i2',
                'iii1','iii3','iii24',
                'hg1','hg2','hg3','hg4',
                'tpd','tpv','ttm','pr']
swarm_names = np.array(trial_db['swarm_names'])
fly_numbers = np.array(trial_db['fly_numbers'])
wb_frequency = np.array(trial_db['wb_frequency'])
trial_names = np.array(trial_db['trial_names'])
wb_selection = np.sum(wb_frequency >50,axis = 1)>1000
swarm_set = [sw for sw in set(swarm_names) if not('pr' in sw)]
fly_set = set(fly_numbers)

/home/psilentp/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:15: RuntimeWarning: invalid value encountered in greater


In [16]:
#calculate the median xcov functions and
#load into a dictionary keyed by group and then muscle
cov_dict = dict()
trial_selection = (trial_names == 'stripe_fix')
wb_selection = np.sum(wb_frequency >50)>1000
wb_selection &= trial_selection
for swarm in swarm_set:
    print swarm
    swarm_selection = (swarm == swarm_names) & wb_selection
    cov_dict[swarm] = dict()
    for mname in muscle_list:
        cov_dict[swarm][mname] = list()
        for flynum in fly_set:
            selection  = (flynum == fly_numbers) & swarm_selection
            if sum(selection) > 2:
                trace = nanmedian(np.array(trial_db['xcov_amp%s'%(mname)])[selection,:],axis = 0)
                cov_dict[swarm][mname].append(trace)               

GMR22H05_GFP
GMR31E10
GMR75B06_GFP
GMR22H05
GMR29E05_GFP
GMR39E01
GMR31E10_GFP
GMR10A12
GMR74F03_GFP
GMR39E01_GFP
GMR75B06
GMR74F03
GMR29E05
GMR10A12_GFP


/home/psilentp/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: RuntimeWarning: invalid value encountered in greater


In [17]:
import h5py
import os
#os.listdir('./')
os.remove('cov_db_sf.hdf5')
cov_db = h5py.File('cov_db_sf.hdf5','a')
for key in swarm_set:
    cov_db.create_group(key)
    for key2 in cov_dict[key].keys():
        cov_db[key].create_dataset(key2,data = np.array(cov_dict[key][key2]),  compression="gzip",compression_opts=5)
cov_db.flush()

In [18]:
swarm_set

['GMR22H05_GFP',
 'GMR31E10',
 'GMR75B06_GFP',
 'GMR22H05',
 'GMR29E05_GFP',
 'GMR39E01',
 'GMR31E10_GFP',
 'GMR10A12',
 'GMR74F03_GFP',
 'GMR39E01_GFP',
 'GMR75B06',
 'GMR74F03',
 'GMR29E05',
 'GMR10A12_GFP']